## Snapcommerce Data Engineer Co-op Case Study

The data below is a stringified table where delimiter is ';' and line_terminator is '\n'.


data = 'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'


### Requirements


Please create a new table with the following transformations on the above command separated data:


Hint: A few libraries are available to help you with this, pandas, pyspark, dask, etc. SQL may be able to help as well. You also don’t need to use any of these. Creative solutions are always welcome.


1. FlightCodes column: Some values are null. Flight Codes are supposed to increase by 10 with each row so 1010 and 1030 will have 1020 in the middle. Fill in these missing numbers and make the column an integer column (instead of a float column).


2. To_From column: Should be split into two separate columns for better analysis! Split on '_' to create two new columns respectively. Also, the case of the column is not very readable, convert the column into capital case.


3. Airline Code column: Clean the Airline Codes to have no punctuation except spaces in the middle. E.g. '(Porter Airways. )' should become 'Porter Airways'.

In [1]:
data = 'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\.Lufthansa.\.""";[12, 33];20055.0;london_MONTreal\n'

In [19]:
import pandas as pd

rows = data.split('\n') # divide by rows
table = [row.split(';') for row in rows[:-1]] # divide by cols, remove last (empty) row

df = pd.DataFrame(table[1:], columns=table[0]) #first row is column headers
df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,(12),[],,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",,Ottawa_VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055.0,london_MONTreal


The following block finds the first valid flight code as a benchmark, storing its value and index. All other values are calculated using offsets from this benchmark.

In [20]:
i = 0
benchmark = None
for code in df["FlightCodes"]:
    if (code is not None and code != ''):
        try:
            benchmark = (float(code), i)
        except ValueError:
            pass
    i += 1
            
if (benchmark is None):
    print("No flight code could be read as float.")
else:
    for j in range(len(df["FlightCodes"])):
        df["FlightCodes"][j] = benchmark[0] + (j-benchmark[1]) * 10
        
df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,(12),[],20025.0,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",20045.0,Ottawa_VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055.0,london_MONTreal


We can use from pandas tricks on Series to make the string changes we want. Finally, we get rid of the old column.

In [21]:
df["To_From"] = df["To_From"].str.upper()
df[["To", "From"]] = df["To_From"].str.split("_", expand=True)
df.drop("To_From", axis=1, inplace=True)
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada (!),"[21, 40]",20015.0,WATERLOO,NEWYORK
1,(12),[],20025.0,MONTREAL,TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALGARY,OTTAWA
3,12. Air France,"[78, 66]",20045.0,OTTAWA,VANCOUVER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055.0,LONDON,MONTREAL


Let's use regex to get rid of everything EXCEPT word characters \w (ie. letters & numbers) and spaces \s 

In [24]:
df["Airline Code"] = df["Airline Code"].str.replace(r'[^\w\s]', '', regex=True)
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015.0,WATERLOO,NEWYORK
1,12,[],20025.0,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035.0,CALGARY,OTTAWA
3,12 Air France,"[78, 66]",20045.0,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055.0,LONDON,MONTREAL


#### And that's it!